In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import ttest_ind
from matplotlib import pyplot as plt
import missingno as mano
import statsmodels.api as sm
from statsmodels.formula.api import ols
from datetime import datetime, timedelta
from sklearn import preprocessing
import pyodbc
from scipy.stats import normaltest, shapiro, chisquare, kstest

In [2]:
# F1: loading data in a dataframe (either CSV or Excel - can be generalized for databases)

def load_data(name,typee):
    if typee=='excel':
        df=pd.read_excel(name)
        return df
    elif typee=='csv':
        df=pd.read_csv(name)
        return df
    elif typee=='sql':
        #query=input('Enter sql query: ')
        conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-FI4K9P1;'
                      'Database=IBA_23387;'
                      'Trusted_Connection=yes;')
        cursor = conn.cursor()
        df = pd.read_sql_query(name,conn)
        return df

In [3]:
# F2: checking shape, column types, and see the first/last 'n' rows using head/tail (where n is one of the arguments of F2)

def df_details(df, n=5):
    x='0'
    print('Shape: ',df.shape)
    print('Column types: ')
    print(df.dtypes)
    while (x!='H' and x!='T'):
        x=input('H/T?')
    if x=='H':
        return df.head(n)
    elif x=='T':
        return df.tail(n)

In [4]:
# F3: remove unnecessary/useless columns (based on results of F2 and your background knowledge and the problem to be solved), e.g., identifiers, multiple primary keys, extra KPI like GMROI in sales which is the same for the whole year etc.

def remove_columns(df, names): # removes columns mentioned in names from df
    for i in names:
        df=df.drop([i], axis=1)
    return df

In [5]:
# F4: remove rows containing a particular value of a given column, e.g., in smoking_status column, I don't want to consider non-smokers in my ML problem so I remove all these rows.

def remove_rows(df, column, names): # removes values from a df column which occur in names
    for i in names:
        df=df.loc[df[column] != i,:]
    return df

In [6]:
# F5: determine the missing values in the whole dataset

def missing_count(df):
    a=pd.DataFrame()
    a['Missing percentage']=(df.isna().sum()/df.shape[0])*100
    a['Missing sum']=df.isna().sum()
    return a

In [7]:
# F6: analyze missing values of one or more columns using mano module

def mano_analysis(*args):
    if len(args)==1:   # if only one argummet is passed, it shows tha mano analysis of all the columns in the dataframe
        df=args[0]
        mano.bar(df)
        mano.matrix(df)
        mano.heatmap(df)
    elif len(args)==2:   # if 2 arguments are passed which includes df as first argumemnt and list of columns as another,
        df=args[0]       # then mano analysis of selected columns are shown
        names=args[1]
        df=df.loc[:,args[1]]
        mano.bar(df)
        mano.matrix(df)
        mano.heatmap(df)

In [8]:
# F7: cater for missing values (input the column with missing value, and the method through which you want to cater for the missing values)

def fill_mv(df,column,method):
    if method=='mean':
        df[column]=df[column].fillna(df[column].mean(),inplace=True)
    elif method=='median':
        df[column]=df[column].fillna(df[column].median(),inplace=True)
    elif method=='mode':
        df[column]=df[column].fillna(df[column].mode(),inplace=True)
    elif method=='value':
        x=input('Enter value: ')
        df[column]=df[column].fillna(x,inplace=True)
        
    elif method=='linear interpolatiom':
        x=input('forward or backward interpolation? ')
        if x=='forward':
            df[column] = df[column].interpolate(method ='linear', limit_direction ='forward')
        elif x=='backward':
            df[column] = df[column].interpolate(method ='linear', limit_direction ='backward')
            
    elif method=='KNN' or method=='knn':
        from sklearn.impute import KNNImputer
        c=int(input('How many columns do you want to impute? '))
        for i in range(c-1):
            y=[column]
            x=input('Enter column: ')
            y.append(x)
            
            
            
        n=int(input('Enter number of neighbors: '))
        imputer = KNNImputer(n_neighbors=n)
        df1=df.copy(deep=True)
        df1[y] = imputer.fit_transform(df1[y])
        
        
        a=input('knn plot column x: ')
        b=input('knn plot column y: ')
        
        nulls=df[a].isna() + df[b].isna()
        
        df1.plot(x=a,y=b,kind='scatter',alpha=0.5,c=nulls,cmap='rainbow')
        #df=df1.copy(deep=True)
        return df1

In [9]:
# F8: Function for numerical data analysis - includes histogram, boxplot, qqplot, describe, and statistical tests for normality

def numerical_analysis(df,x):
    print(df[x].describe())
    print('')
    
    print('According to skew and kurtosis test: ')
    Skew=skew(df[x])
    Kurtosis=kurtosis(df[x])
    print('Skew = ',Skew)
    print('Kurtosis = ',Kurtosis)
    stat, p=normaltest(df[x])
    print('stat= ',stat)
    print('p= ',p)
    if p<0.05:
        print('Probably not Normal')
    else:
        print('Probably Normal')
        
    print('')
    print('According to Shapiro-Wilk test:')
    stat,p=shapiro(df[x])
    print('stat= ',stat)
    print('p= ',p)
    if p<0.05:
        print('Probably not Normal')
    else:
        print('Probably Normal')
        
    print('')
    print('According to Chi-Squared normality test: ')
    stat, p=chisquare(df[x])
    print('stat= ',stat)
    print('p= ',p)
    if p<0.05:
        print('Probably not Normal')
    else:
        print('Probably Normal')
        
    print('')
    print('according to Kolmogorov-Smirnov test: ')
    stat,p=kstest(df[x],'norm')
    print('stat= ',stat)
    print('p= ',p)
    if p<0.05:
        print('Probably not Normal')
    else:
        print('Probably Normal')
    
    
    
    sns.distplot(df[x])
    plt.figure(figsize=(10,8))
    sns.boxplot(df[x])
    fig = sm.qqplot(df[x], line='45')
    plt.show()

In [10]:
# F9: Function for categorical data analysis - includes value counts, and bar charts

def categorical_count(df,x):  # prints value_counts of categorical columns and also plots the bar chart against the count
    print(df[x].value_counts())
    
    aa=pd.DataFrame(df[x].value_counts())
    aa=aa.rename(columns={x:'COUNT'})
    aa[x]=aa.index
    plt.bar(aa[x],aa['COUNT'])
    plt.xlabel(x)
    plt.ylabel('Count')
    
def categorical_analysis(df,x,KPI,aggfunc):  # plots bar chart of KPI against a categorical column. For eg price against
    z=pd.pivot_table(df,index=x,values=KPI,aggfunc=aggfunc)  # product type
    z[x]=z.index
    plt.bar(z[x],z[KPI])
    plt.xlabel(x)
    plt.ylabel(KPI)

In [11]:
# F10: Function to change the type of any column (input col name and the type you want)

def to_numeric(df,columns):
    for i in columns:
        df[i]=pd.to_numeric(df[i])
    return df[i]
def to_str(df,columns):
    for i in columns:
        df[i]=df[i].astype(str)
    return df[i]
def to_datetime(df, columns):
    for i in columns:
        df[i]=pd.to_datetime(df[i])
    return df[i]

In [12]:
# F11: Function to change the discretizations of a particular categorical column, e.g., rename the values, remove space between value names etc

def strip(df, column): # removes leading and trailing white spaces. Also removes excess white spaces between words
    import re
    df[column]=df[column].str.strip()
    df[column] = df[column].replace('\s+', ' ', regex=True)
    
def discretize(df,column,value1, value2): # replaces value1 with value2.
    df[column].replace(value1, value2, inplace=True) # value1 could be just one string value or list of string values
    return df[column]

In [13]:
# F12: Function for date analysis - extract year, month etc., subtract dates etc. (this function cannot be specified exactly so just add what you believe are the basic things)

# import datetime module for this function
def date_extract(df,column): 
    x=input('Would you like to extract year(Y),month(M),day(D),week(W),hour(H),minute(min) or second(S)?: ')
    if (x=='Y') or (x=='y'):
        return df[column].dt.year
    elif (x=='M') or (x=='m'):
        return df[column].dt.month
    elif (x=='D') or (x=='d'):
        return df[column].dt.day
    elif (x=='W') or (x=='w'):
        return df[column].dt.week
    elif (x=='H') or (x=='h'):
        return df[column].dt.hour
    elif (x=='min'):
        return df[column].dt.minute
    else:
        return df[column].dt.second

# import timedelta module for following function
def date_algebra(df, column, days, op):
    if op=='sum':
        return (df[column]+timedelta(days=days))
    elif op=='subtract':
        return (df[column]-timedelta(days=days))
    else:
        print('op should be either sum or subtract')
        
def duration(df, column1,column2): # return duration between two date columns in days
    d=df[column2]-df[column1]
    x=input('Would you like duration in years(Y),months(M),days(D),weeks(W),hours(H),minutes(min) or seconds(S)?: ')
    return d

In [14]:
# F13: function to make a deep copy of a dataframe

def df_copy(df):
    return df.copy(deep=True)

In [15]:
# F14: function to encode categorical into numerical (label, ordinal, or onehot)

def encode(df, column, encoder, ordinal_dict={}):
    if encoder=='label':
        label_encoder = preprocessing.LabelEncoder()
        df[column]=label_encoder.fit_transform(df[[column]])
        return df
    elif (encoder=='one hot') or (encoder=='onehot'):
        #prefix=input('What prefix do you want for your column?: ')
        df=pd.get_dummies(df, columns=column)
        return df
    elif encoder=='ordinal':
        df[column]=df[column].map(ordinal_dict) # {'Male':1,'Female':0}
        return df
    else:
        print('Encoder argument should be either label, ordinal or one hot/onehot.')

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score, log_loss, mean_absolute_error, mean_squared_error, r2_score

In [17]:
# F15: function to split dataframe into X (predictors) and y (label), apply standard scaling on X, apply the desired ML algorithm and output the results:

def ml(df,label,algo,classification=True):
    oversample=input('Do you want to oversample your training data? (0/1): ')
    if algo=='KNN':
        n=int(input('Enter number of neighbors: '))
    P=df.drop([label],axis=1)
    L=df[label]
    scaler=StandardScaler()
    for i in P.columns:
        P[i]=scaler.fit_transform(P[[i]])
    dict_classifier={'Logistic Regression':LogisticRegression(),'Decision Tree':DecisionTreeClassifier(),'KNN':KNeighborsClassifier(n_neighbors=n),
                     'Naive Bayes':GaussianNB(),'Random Forrest':RandomForestClassifier(),'Gradient Boosting':GradientBoostingClassifier()}
    dict_regressor={'Linear Regression':LinearRegression(),'Decision Tree':DecisionTreeRegressor(),'KNN':KNeighborsRegressor(n_neighbors=n),
                    'Random Forest':RandomForestRegressor(),'Gradient Boosting':GradientBoostingRegressor()}

    ts=float(input('What do you want your test size to be? ')) #0.2
    x_train, x_test, y_train, y_test = train_test_split(P,L,test_size=ts,random_state=42)

    if classification ==True:
        if oversample=='1':
            from imblearn.over_sampling import RandomOverSampler
            sample = RandomOverSampler(sampling_strategy=0.5)
            x_train, y_train=sample.fit_resample(x_train,y_train)
        classifier=dict_classifier[algo]
        classifier.fit(x_train, y_train)
        y_pred=classifier.predict(x_test)
    
        print('Classification report: ')
        print(classification_report(y_test,y_pred))
        print('Accuracy(%): ')
        print(accuracy_score(y_test,y_pred)*100)
        print('Precision(%): ')
        print(precision_score(y_test,y_pred)*100)
        print('F1 score: ')
        print(f1_score(y_test,y_pred))
        print('Logistic loss: ')
        print(log_loss(y_test,y_pred))
    
        confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
        sns.heatmap(confusion_matrix, annot=True)
        plt.show()
    
    else:
        regressor=dict_regressor[algo]
        regressor.fit(x_train, y_train)
        y_pred=regressor.predict(x_test)
        
        plt.scatter(y_test, y_pred, c = 'green') 
        plt.xlabel("True Value") 
        plt.ylabel("Predicted value") 
        plt.title("True value vs predicted value") 
        plt.show()
    
        print('Mean Absolute Error: ')
        print(mean_absolute_error(y_test,y_pred))
        print('Mean Squared Error: ')
        print(mean_squared_error(y_test,y_pred))
        print('R squared error: ')
        print(r2_score(y_test,y_pred))
                

In [18]:
def error_rate_plot(df, label, classification=True, oversample=False):
    P=df.drop([label],axis=1)
    L=df[label]
    scaler=StandardScaler()
    for i in P.columns:
        P[i]=scaler.fit_transform(P[[i]])
    
    ts=float(input('What do you want your test size to be? ')) #0.2
    x_train, x_test, y_train, y_test = train_test_split(P,L,test_size=ts,random_state=42)
    if oversample==True:
        from imblearn.over_sampling import RandomOverSampler
        sample = RandomOverSampler(sampling_strategy=0.5)
        x_train, y_train=sample.fit_resample(x_train,y_train)
    error=[]
    if classification==True:
        for i in range(1,35):
            classifier=KNeighborsClassifier(n_neighbors=i)
            classifier.fit(x_train, y_train)
            y_pred=classifier.predict(x_test)
            error.append(np.mean(y_pred!=y_test))
        plt.figure(figsize=(12,6))
        plt.plot(range(1,35),error,color='red',marker='o',markersize=10)
        plt.title('Error rate K value')
        plt.xlabel('K value')
        plt.ylabel('Mean Error')
            
    else:
        for i in range(1,35):
            regressor=KNeighborsRegressor(n_neighbors=i)
            regressor.fit(x_train, y_train)
            y_pred=regressor.predict(x_test)
            error.append(np.mean(y_pred!=y_test))
        plt.figure(figsize=(12,6))
        plt.plot(range(1,35),error,color='red',marker='o',markersize=10)
        plt.title('Error rate K value')
        plt.xlabel('K value')
        plt.ylabel('Mean Error')

Try the following with no missing values

In [19]:
# F16: Function to apply ANOVA and output results

def anova_test(df):
    x=input('Enter the KPI: ')
    y=input('Enter the categorical column: ')
    model = ols(x + ' ~ C(Q('+y+'))', data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print ("\nAnova => "+x+" - "+y)
    display(anova_table)

In [20]:
# F17: Function to generate correlation heatmaps

def corr_heatmap(df):
    corrmatrix = df.corr()
    f, axis = plt.subplots(figsize =(15, 10)) 
    sns.heatmap(corrmatrix, ax = axis, linewidths = 0.2,annot=True)

In [21]:
# F18: Function to generate scatter plot

def scatter(df, column1, column2):
    plt.scatter(df[column1], df[column2], c='green')
    plt.xlabel(column1)
    plt.ylabel(column2)
    plt.show()